## Implementation of Camera Calibration through OpenCV

In [1]:
# Imports
import cv2
import numpy as np
import glob

## Define dimensions of checkerboard

In [2]:
# Defining the dimensions of checkerboard
 # 9,6 to begin with

# Each tile is 7x7 mm
chekcerboard_tile_size = 7 #mm

CHECKERBOARD = (4,4) # Vertical, horizontal
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 

## Define world coordinates for 3D points

In [3]:
# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

## Extract path of individual image stored in ChessBoards folder

In [4]:
# Need to calibrate with more than one image ideally. 
images = glob.glob('./Images/ChessBoards/LimitedBoards4/*.bmp') 
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
     
    """
    If desired number of corner are detected, we refine the pixel coordinates and display them on the images of checker board
    """
    
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
         
        imgpoints.append(corners2)
 
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
     
    cv2.imshow('img',img)
    cv2.waitKey(0)
 
cv2.destroyAllWindows()

h,w = img.shape[:2]
 

## Camera calibration using OpenCV
* Zhang based calibration


In [5]:
"""
Performing camera calibration by passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the detected corners (imgpoints)
"""

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("Camera matrix : \n") # Intrinsic parameters
print(mtx)
print("\n dist : \n") # [k1, k2, p1, p2, k3] k:radial distortion, p: tangential distortion
print(dist)
print("\n rvecs : \n") # Extrinsic Rodriguez
print(rvecs)
print("\n tvecs : \n") # Extrinsic Translation vector
print(tvecs)

print("tvecs[0] ", tvecs[0])
'''Constrain the world to a plane, simplified to a 3x3 matrix.
Projection between two planes, world and sensor. So when we project, we don't lose information.

H = lambda * [intrinsic]*[extrinsic]
alpha = focal length in pixels

point in world (P_w) = inv(Homography)* point in pixels(P_d)

Project 3D point [0,0,0] to image, [100, 0,0 ] draw a line, which projects the X of the world image???
'''



Camera matrix : 

[[4.13922090e+03 0.00000000e+00 3.22320257e+02]
 [0.00000000e+00 4.18974476e+03 2.45865771e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

 dist : 

[[-4.79969522e+00  3.10243894e+03 -2.01999506e-02 -2.33813766e-02
   1.27655739e+01]]

 rvecs : 

(array([[-0.15064384],
       [ 0.01118792],
       [-0.00286632]]), array([[-0.15712447],
       [ 0.03586678],
       [ 0.38804825]]), array([[ 0.13030881],
       [ 0.01566151],
       [-0.03903251]]), array([[ 0.14234059],
       [ 0.03137642],
       [-0.14062137]]), array([[ 0.12668345],
       [-0.00510366],
       [ 0.2021847 ]]))

 tvecs : 

(array([[-1.22658955],
       [-0.91221106],
       [69.49746313]]), array([[-0.5243562 ],
       [-1.97960022],
       [69.59578979]]), array([[-1.2827288 ],
       [-0.80550459],
       [70.51531702]]), array([[-1.46670374],
       [-0.69734627],
       [70.44735871]]), array([[-1.03732726],
       [-1.28759936],
       [70.47840352]]))
tvecs[0]  [[-1.22658955]
 [-0.9122

"Constrain the world to a plane, simplified to a 3x3 matrix.\nProjection between two planes, world and sensor. So when we project, we don't lose information.\n\nH = lambda * [intrinsic]*[extrinsic]\nalpha = focal length in pixels\n\npoint in world (P_w) = inv(Homography)* point in pixels(P_d)\n\nProject 3D point [0,0,0] to image, [100, 0,0 ] draw a line, which projects the X of the world image???\n"

In [6]:
# Invert camera matrix:
np.linalg.inv(mtx)
print("Inverted camera matrix: \n")
print(mtx)

# 

Inverted camera matrix: 

[[4.13922090e+03 0.00000000e+00 3.22320257e+02]
 [0.00000000e+00 4.18974476e+03 2.45865771e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


## Converting Rodrigues to rotational matrix

In [7]:
rotation_matrix = np.zeros(shape=(3,3))
cv2.Rodrigues(rvecs[0], rotation_matrix)
print("Rotation matrix: \n", rotation_matrix)

rotation_mask = np.array([[1, 0], [0, 1], [0, 0]])

chopped_rotation = np.matmul(rotation_matrix, rotation_mask)
# print("\nChopped matrix: \n", chopped_rotation)

extrinsic_matrix = np.hstack((chopped_rotation,tvecs[0]))
# print("\nExtrinsic matrix: \n", extrinsic_matrix)

homography_matrix = np.matmul(mtx, extrinsic_matrix)
print("\nPre homography matrix: \n", homography_matrix)

homography_matrix = homography_matrix/homography_matrix[2][2]
print("\nPost homography matrix: \n", homography_matrix)

inv_homography_matrix = np.linalg.inv(homography_matrix)
print("\n Inverse homography matrix: \n", inv_homography_matrix)

pixels = np.array((1,2,1))
pixels = pixels.reshape(1,-1).transpose()
print("Pixels: \n", pixels)

world_coordinates = inv_homography_matrix * pixels
print("\nWorld Coordinates: \n", world_coordinates)


Rotation matrix: 
 [[ 0.99993343  0.00201433  0.01136089]
 [-0.00369652  0.98867068  0.15005537]
 [-0.01092992 -0.15008738  0.98861232]]

Pre homography matrix: 
 [[ 4.13542244e+03 -4.00384240e+01  1.73233151e+04]
 [-1.81747812e+01  4.10537647e+03  1.32651158e+04]
 [-1.09299202e-02 -1.50087376e-01  6.94974631e+01]]

Post homography matrix: 
 [[ 5.95046532e+01 -5.76113461e-01  2.49265431e+02]
 [-2.61517188e-01  5.90723213e+01  1.90871943e+02]
 [-1.57270779e-04 -2.15960941e-03  1.00000000e+00]]

 Inverse homography matrix: 
 [[ 1.67943857e-02  1.06714406e-05 -4.18829665e+00]
 [ 6.53594626e-05  1.68111351e-02 -3.22506587e+00]
 [ 2.78241703e-06  3.63071639e-05  9.92376421e-01]]
Pixels: 
 [[1]
 [2]
 [1]]

World Coordinates: 
 [[ 1.67943857e-02  1.06714406e-05 -4.18829665e+00]
 [ 1.30718925e-04  3.36222701e-02 -6.45013174e+00]
 [ 2.78241703e-06  3.63071639e-05  9.92376421e-01]]


In [14]:
# Function for legible outputs:

def calculateWorldCoordinates(pixels_to_calculate, intrinsicMatrix, extrinsicMatrix):
    inverse_intrinsic = np.linalg.inv(intrinsicMatrix)
    inverse_extrinsic = np.linalg.inv(extrinsicMatrix)
    init_coords = np.matmul(inverse_extrinsic,inverse_intrinsic)
    world_coordinate_vector = np.matmul(init_coords, pixels_to_calculate)
    return world_coordinate_vector

'''
(Retrieve pixel coordinate from image.)
Reshape pixels
Calculate to world coordinates.
Print world coordinates.
Print world coordinates to csv.
(Graphical presentation??)
'''

pix1 = np.array((1,2,1))
pix2 = np.array((2,3,4))
pix3 = np.array((5,5,5))
pixels = [pix1, pix2, pix3]

def coordinatesToCSV(pixels_to_calculate, intrinsicMatrix, extrinsicMatrix, outputFile):
    # pixelCoordinates = pixelCoordinates.reshape(1,-1).transpose()
    ok = True
    try: 
        outputFile = open(outputFile, "w")
    except:
        ok = False
        print("Unable to open output file.")
    if ok:
        outputFile.write("World coordinates are on the form [X, Y, Z, 1], where the '1' is left out for this file.\n")
        for pixelCount,pixelCoordinate in enumerate(pixels_to_calculate):
            outputFile.write("\nPixel count: " + str(pixelCount) + "\nPixel coordinates: " + str(pixelCoordinate) + "\n")
            worldCoordinates = calculateWorldCoordinates(pixelCoordinate, intrinsicMatrix, extrinsicMatrix )
            outputFile.write("\nWorld coordinates: \n" + str(worldCoordinates) + "\n")
    
    print("Written to file: ", outputFile)
    outputFile.close()

outputFile = "Coordinates.csv"
coordinatesToCSV(pixels, mtx, extrinsic_matrix, outputFile)

Written to file:  <_io.TextIOWrapper name='Coordinates.csv' mode='w' encoding='cp1252'>
